In [49]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [50]:
imdb_reviews=pd.read_csv("/content/imdb_reviews.csv")
test_reviews=pd.read_csv("/content/test_reviews.csv")
imdb_reviews.head(10)

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative
5,<START begins better than it ends funny that t...,negative
6,<START lavish production values and solid perf...,positive
7,<START the <UNK> tells the story of the four h...,negative
8,<START just got out and cannot believe what a ...,positive
9,<START this movie has many problem associated ...,negative


In [51]:
test_reviews.head(10)

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive
5,<START i'm absolutely disgusted this movie isn...,positive
6,<START originally supposed to be just a part o...,positive
7,<START the <UNK> richard <UNK> dog is <UNK> to...,negative
8,<START hollywood had a long love affair with b...,negative
9,<START this film is where the batman franchise...,positive


In [52]:
word_index=pd.read_csv("/content/word_indexes.csv")
word_index.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


In [53]:
word_index=dict(zip(word_index.Words,word_index.Indexes))
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3


In [54]:
def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr

In [55]:
train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data, test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

In [56]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

In [57]:

train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)
train_data.head()

0    [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...
1    [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...
2    [1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...
3    [1, 4, 2, 2, 33, 2804, 4, 2040, 432, 111, 153,...
4    [1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...
Name: Reviews, dtype: object

In [58]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)


In [59]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [60]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

In [61]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [62]:
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 3s 54ms/step - loss: 0.6918 - accuracy: 0.5378 - val_loss: 0.6893 - val_accuracy: 0.7130
Epoch 2/30
49/49 [==============================] - 3s 57ms/step - loss: 0.6820 - accuracy: 0.6469 - val_loss: 0.6735 - val_accuracy: 0.6840
Epoch 3/30
49/49 [==============================] - 2s 43ms/step - loss: 0.6547 - accuracy: 0.7272 - val_loss: 0.6388 - val_accuracy: 0.7522
Epoch 4/30
49/49 [==============================] - 2s 44ms/step - loss: 0.6084 - accuracy: 0.7811 - val_loss: 0.5896 - val_accuracy: 0.7972
Epoch 5/30
49/49 [==============================] - 2s 47ms/step - loss: 0.5489 - accuracy: 0.8188 - val_loss: 0.5327 - val_accuracy: 0.8199
Epoch 6/30
49/49 [==============================] - 2s 41ms/step - loss: 0.4868 - accuracy: 0.8434 - val_loss: 0.4785 - val_accuracy: 0.8349
Epoch 7/30
49/49 [==============================] - 2s 43ms/step - loss: 0.4303 - accuracy: 0.8631 - val_loss: 0.4333 - val_accuracy: 0.8490
Epoch 8/30
49

In [63]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 2s 3ms/step - loss: 0.2947 - accuracy: 0.8855


In [64]:
index=np.random.randint(1,1000)
user_review=test_reviews.loc[index]
print(user_review)


Reviews      <START fairly appalling enterprise suggests <U...
Sentiment                                             negative
Name: 34, dtype: object


In [65]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review)>0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")

1/1 [==============================] - 0s 69ms/step
negative sentiment


Now we can use this model, that is able to detection the sentiment of the text and feedback.
